<a href="https://colab.research.google.com/github/gr3ybr0w/cookbook/blob/master/Machine_Learning/Neural_Networks/TensorFlow/Keras/womensEcomReviews.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
# making the move as kaggle now looks in a different location for the file
!cp /content/.kaggle/kaggle.json /root/.kaggle/kaggle.json
!pip install kaggle

cp: cannot stat '/content/.kaggle/kaggle.json': No such file or directory


In [23]:
!ls -la /.kaggle/kaggle.json

total 12
drwxr-xr-x 2 root root 4096 Feb  5 15:17 .
drwxr-xr-x 1 root root 4096 Feb  5 15:17 ..
---x-wx--T 1 root root   70 Feb  5 15:18 kaggle.json


In [24]:
# making the move as kaggle now looks in a different location for the file
!cp /.kaggle/kaggle.json /root/.kaggle/kaggle.json

from googleapiclient.discovery import build
import io, os
from googleapiclient.http import MediaIoBaseDownload
from google.colab import auth
auth.authenticate_user()
drive_service = build('drive', 'v3')
results = drive_service.files().list(
        q="name = 'kaggle.json'", fields="files(id)").execute()
kaggle_api_key = results.get('files', [])
filename = "/.kaggle/kaggle.json"
os.makedirs(os.path.dirname(filename), exist_ok=True)
request = drive_service.files().get_media(fileId=kaggle_api_key[0]['id'])
fh = io.FileIO(filename, 'wb')
downloader = MediaIoBaseDownload(fh, request)
done = False
while done is False:
    status, done = downloader.next_chunk()
    print("Download %d%%." % int(status.progress() * 100))
os.chmod(filename, 600)

Download 100%.


In [25]:
!kaggle datasets download -d nicapotato/womens-ecommerce-clothing-reviews

  0% 0.00/2.79M [00:00<?, ?B/s]
100% 2.79M/2.79M [00:00<00:00, 92.5MB/s]


In [26]:
!ls

adc.json  sample_data  womens-ecommerce-clothing-reviews.zip


In [27]:
!unzip womens-ecommerce-clothing-reviews.zip

Archive:  womens-ecommerce-clothing-reviews.zip
  inflating: Womens Clothing E-Commerce Reviews.csv  


In [28]:
!ls

 adc.json     'Womens Clothing E-Commerce Reviews.csv'
 sample_data   womens-ecommerce-clothing-reviews.zip


In [0]:
import pandas as pd
import numpy as np
import os

In [30]:
data = pd.read_csv('./Womens Clothing E-Commerce Reviews.csv').drop(['Unnamed: 0'], axis = 1)
data = data.dropna()
data.head()

,Clothing ID,Age,Title,Review Text,Rating,Recommended IND,Positive Feedback Count,Division Name,Department Name,Class Name
2,1077,60,Some major design flaws,I had such high hopes for this dress and reall...,3,0,0,General,Dresses,Dresses
3,1049,50,My favorite buy!,"I love, love, love this jumpsuit. it's fun, fl...",5,1,0,General Petite,Bottoms,Pants
4,847,47,Flattering shirt,This shirt is very flattering to all due to th...,5,1,6,General,Tops,Blouses
5,1080,49,Not for the very petite,"I love tracy reese dresses, but this one is no...",2,0,4,General,Dresses,Dresses
6,858,39,Cagrcoal shimmer fun,I aded this in my basket at hte last mintue to...,5,1,1,General Petite,Tops,Knits


In [0]:
X, y = data.drop(['Class Name'], axis = 1), data['Class Name']

In [32]:
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.layers import Embedding, GlobalMaxPooling1D
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.utils import to_categorical
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

Using TensorFlow backend.


In [0]:
title_tokenizer = Tokenizer()
text_tokenizer = Tokenizer()
title_tokenizer.fit_on_texts(X['Title'].values)
text_tokenizer.fit_on_texts(X['Review Text'])

In [0]:
from sklearn.model_selection import train_test_split
X_train, X_validation, y_train, y_validation = train_test_split(
    X, y, test_size = 0.2
)

In [0]:
train_title = title_tokenizer.texts_to_sequences(X_train['Title'].values)
train_text = text_tokenizer.texts_to_sequences(X_train['Review Text'].values)
validation_title = title_tokenizer.texts_to_sequences(X_validation['Title'].values)
validation_text  = text_tokenizer.texts_to_sequences(X_validation['Review Text'].values)

In [0]:
title_vocab_size = len(title_tokenizer.word_index) + 1
text_vocab_size  = len(text_tokenizer.word_index) + 1

In [0]:
title_max_length = 300
text_max_length  = 1000

In [0]:
train_title = pad_sequences(train_title, padding = 'post', maxlen = title_max_length)
train_text  = pad_sequences(train_text, padding = 'post', maxlen = text_max_length)
validation_title = pad_sequences(validation_title, padding = 'post', maxlen = title_max_length)
validation_text  = pad_sequences(validation_text, padding = 'post', maxlen = text_max_length)

## Predicting with only Title

In [39]:
model = Sequential()
model.add(Embedding(input_dim = title_vocab_size, output_dim = 100, input_length = title_max_length))
model.add(GlobalMaxPooling1D())
model.add(Dense(64, activation = 'relu'))
model.add(Dense(64, activation = 'relu'))
model.add(Dropout(0.2))
model.add(Dense(32, activation = 'relu'))
model.add(Dense(32, activation = 'relu'))
model.add(Dropout(0.2))
model.add(Dense(20, activation = 'sigmoid'))

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [0]:
model.compile(
    loss = 'categorical_crossentropy',
    optimizer = 'adam',
    metrics = ['accuracy']
)

In [0]:
from sklearn.preprocessing import LabelEncoder

In [0]:
label_encoder = LabelEncoder().fit(y.values)

In [0]:
y_train = to_categorical(label_encoder.transform(y_train), num_classes = 20)
y_validation = to_categorical(label_encoder.transform(y_validation), num_classes = 20)

In [44]:
model.fit(train_title, y_train, validation_data = (validation_title, y_validation), epochs = 100, verbose = 1, callbacks = [
    ModelCheckpoint(
        'title_model.h5',
        monitor = 'val_acc',
        mode = 'max'
    ),
    EarlyStopping(
        monitor = 'val_acc',
        mode = 'max'
    )
])

Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
Train on 15729 samples, validate on 3933 samples
Epoch 1/100
15729/15729 [==============================] - 6s 411us/step - loss: 2.3662 - acc: 0.2607 - val_loss: 2.1330 - val_acc: 0.2883
Epoch 2/100
15729/15729 [==============================] - 4s 262us/step - loss: 2.0877 - acc: 0.2893 - val_loss: 1.9937 - val_acc: 0.2998
Epoch 3/100
15729/15729 [==============================] - 4s 261us/step - loss: 1.9317 - acc: 0.3168 - val_loss: 1.9516 - val_acc: 0.3285
Epoch 4/100
15729/15729 [==============================] - 4s 261us/step - loss: 1.8157 - acc: 0.3694 - val_loss: 1.9346 - val_acc: 0.3758
Epoch 5/100
15729/15729 [==============================] - 4s 261us/step - loss: 1.6991 - acc: 0.4265 - val_loss: 1.9143 - val_acc: 0.3888
Epoch 6/100
15729/15729 [==============================] - 4s 261us/step - loss: 1.5969 - acc: 0.4581 - val_loss: 1.9457 - val_acc

In [0]:
from keras.models import load_model

In [0]:
model = load_model('title_model.h5')

In [47]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 300, 100)          382600    
_________________________________________________________________
global_max_pooling1d_1 (Glob (None, 100)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 64)                6464      
_________________________________________________________________
dense_2 (Dense)              (None, 64)                4160      
_________________________________________________________________
dropout_1 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 32)                2080      
_________________________________________________________________
dense_4 (Dense)              (None, 32)                1056      
__________

In [48]:
train_loss, train_accuracy = model.evaluate(train_title, y_train)
validation_loss, validation_accuracy = model.evaluate(validation_title, y_validation)
print("Train :", train_accuracy)
print("Validation :", validation_accuracy)

3933/3933 [==============================] - 0s 56us/step
Train : 0.4992052895943672
Validation : 0.38189677084459167


## Predicting with only Review Text

In [0]:
model = Sequential()
model.add(Embedding(input_dim = text_vocab_size, output_dim = 512, input_length = text_max_length))
model.add(GlobalMaxPooling1D())
model.add(Dense(256, activation = 'relu'))
model.add(Dense(256, activation = 'relu'))
model.add(Dropout(0.2))
model.add(Dense(128, activation = 'relu'))
model.add(Dense(128, activation = 'relu'))
model.add(Dropout(0.2))
model.add(Dense(20, activation = 'sigmoid'))

In [0]:
model.compile(
    loss = 'categorical_crossentropy',
    optimizer = 'adam',
    metrics = ['accuracy']
)

In [51]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 1000, 512)         7293440   
_________________________________________________________________
global_max_pooling1d_2 (Glob (None, 512)               0         
_________________________________________________________________
dense_6 (Dense)              (None, 256)               131328    
_________________________________________________________________
dense_7 (Dense)              (None, 256)               65792     
_________________________________________________________________
dropout_3 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_8 (Dense)              (None, 128)               32896     
_________________________________________________________________
dense_9 (Dense)              (None, 128)               16512     
__________

In [52]:
model.fit(train_text, y_train, validation_data = (validation_text, y_validation), epochs = 100, verbose = 1, callbacks = [
    ModelCheckpoint(
        'text_model.h5',
        monitor = 'val_acc',
        mode = 'max'
    ),
    EarlyStopping(
        monitor = 'val_acc',
        mode = 'max'
    )
])

Train on 15729 samples, validate on 3933 samples
Epoch 1/100
15729/15729 [==============================] - 12s 761us/step - loss: 1.9503 - acc: 0.3334 - val_loss: 1.3647 - val_acc: 0.5411
Epoch 2/100
15729/15729 [==============================] - 11s 692us/step - loss: 1.2843 - acc: 0.5641 - val_loss: 1.1983 - val_acc: 0.5830
Epoch 3/100
15729/15729 [==============================] - 11s 689us/step - loss: 1.0615 - acc: 0.6218 - val_loss: 1.1583 - val_acc: 0.5998
Epoch 4/100
15729/15729 [==============================] - 11s 689us/step - loss: 0.8883 - acc: 0.6728 - val_loss: 1.2190 - val_acc: 0.6156
Epoch 5/100
15729/15729 [==============================] - 11s 689us/step - loss: 0.7203 - acc: 0.7418 - val_loss: 1.3601 - val_acc: 0.6074


In [0]:
model = load_model('text_model.h5')

In [54]:
train_loss, train_accuracy = model.evaluate(train_text, y_train)
validation_loss, validation_accuracy = model.evaluate(validation_text, y_validation)
print("Train :", train_accuracy)
print("Validation :", validation_accuracy)

3933/3933 [==============================] - 0s 105us/step
Train : 0.8074893508956971
Validation : 0.6074243580873704


## Predicting with full data

In [0]:
X_train = X_train.drop(['Clothing ID'], axis = 1)
X_validation = X_validation.drop(['Clothing ID'], axis = 1)

In [0]:
X_train, X_validation = (
    X_train.drop(['Title', 'Review Text'], axis = 1),
    X_validation.drop(['Title', 'Review Text'], axis = 1)
)

In [0]:
X_train['Division Name'] = LabelEncoder().fit(X['Division Name']).transform(X_train['Division Name'])
X_validation['Division Name'] = LabelEncoder().fit(X['Division Name']).transform(X_validation['Division Name'])

In [0]:
X_train['Department Name'] = LabelEncoder().fit(X['Department Name']).transform(X_train['Department Name'])
X_validation['Department Name'] = LabelEncoder().fit(X['Department Name']).transform(X_validation['Department Name'])

In [0]:
from keras.models import Model
from keras.layers import Input
from keras.layers import concatenate

def _text_block(x, vocab_size, base_num = 512):
    x = Embedding(input_dim = vocab_size, output_dim = base_num)(x)
    x = GlobalMaxPooling1D()(x)
    x = Dense(base_num // 2, activation = 'relu')(x)
    x = Dense(base_num // 2, activation = 'relu')(x)
    x = Dropout(0.2)(x)
    x = Dense(base_num // 4, activation = 'relu')(x)
    x = Dense(base_num // 4, activation = 'relu')(x)
    return x

def _model():
    basic_input = Input(shape = (6, ))
    title_input = Input(shape = (300, ))
    text_input  = Input(shape = (1000, ))
    basic_x = Dense(6, activation = 'relu')(basic_input)
    title_x = _text_block(title_input, vocab_size = title_vocab_size, base_num = 128)
    text_x  = _text_block(text_input, vocab_size = text_vocab_size, base_num = 512)
    x = concatenate([basic_x, title_x, text_x])
    x = Dense(512, activation = 'relu')(x)
    x = Dense(256, activation = 'relu')(x)
    x = Dropout(0.2)(x)
    x = Dense(128, activation = 'relu')(x)
    x = Dense(64, activation = 'relu')(x)
    x = Dropout(0.2)(x)
    output = Dense(20, activation = 'sigmoid')(x)
    return Model(inputs = [basic_input, title_input, text_input], outputs = output)

In [0]:
model = _model()

In [0]:
model.compile(
    loss = 'categorical_crossentropy',
    optimizer = 'adam',
    metrics = ['accuracy']
)

In [62]:
model.fit(
    [X_train.values, train_title, train_text],
    y_train,
    validation_data = (
        [X_validation.values, validation_title, validation_text],
        y_validation
    ),
    epochs = 100,
    callbacks = [
        ModelCheckpoint(
            'model.h5',
            monitor = 'val_acc',
            mode = 'max'
        ),
        EarlyStopping(
            monitor = 'val_acc',
            mode = 'max'
        )
    ]
)

Train on 15729 samples, validate on 3933 samples
Epoch 1/100
15729/15729 [==============================] - 16s 988us/step - loss: 2.0468 - acc: 0.3240 - val_loss: 1.2864 - val_acc: 0.5161
Epoch 2/100
15729/15729 [==============================] - 14s 898us/step - loss: 1.1278 - acc: 0.5534 - val_loss: 0.9879 - val_acc: 0.5848
Epoch 3/100
15729/15729 [==============================] - 14s 896us/step - loss: 0.8550 - acc: 0.6322 - val_loss: 0.7780 - val_acc: 0.6593
Epoch 4/100
15729/15729 [==============================] - 14s 896us/step - loss: 0.7320 - acc: 0.6707 - val_loss: 0.7778 - val_acc: 0.6573


In [0]:
model = load_model('model.h5')

In [64]:
train_loss, train_accuracy = model.evaluate([X_train.values, train_title, train_text], y_train)
validation_loss, validation_accuracy = model.evaluate([X_validation.values, validation_title, validation_text], y_validation)
print("Train :", train_accuracy)
print("Validation :", validation_accuracy)

3933/3933 [==============================] - 1s 136us/step
Train : 0.7034776527660772
Validation : 0.6572590897079039
